Data

In [1]:
for filename in ["vocab", "sentences.sample", "entityID2contexts"]:
    path = "./data/" + filename + ".txt"
    f = open(path, "r", encoding="utf-8")
    print('\n\n---------' + filename + '---------')
    print(f.readline())
    print(f.readline())
    f.close()
print("* EntityID  SentenceID(context)  weight")



---------vocab---------
EntityID	PreferredName	AllNames (seperated by '||')

1	learning	learning



---------sentences.sample---------
SentenceID	Sentence

aa-0#0	Distributed attention .



---------entityID2contexts---------
8519	aa-0#1	1

7	aa-0#1	1

* EntityID  SentenceID(context)  weight


Model

In [2]:
# DIH
from preprocess import *

from patternBased.preprocess import read_sparse_matrix
from DIH import baseDIH, SLQS, SLQS_Cos

vocab_path="./data/vocab.txt"
predict_pairs_path="./data/test_pairs.txt"
predict_pairs=read_predict_pairs(vocab_path, predict_pairs_path)

spacefile_path="./data/entityID2contexts.txt"
basedih=baseDIH(spacefile_path)
slqs=SLQS(spacefile_path, topk=50)
slqs_cos=SLQS_Cos(spacefile_path, topk=50)

100%|████████████████████████████████████████████████████████████████████████████████| 533/533 [01:19<00:00,  6.69it/s]


In [3]:
write_model(basedih, "baseDIH")
write_model(slqs, "slqs")
write_model(slqs_cos, "slqs_cos")

Prediction

In [4]:
def _predict(model=None, metric_name=None):
    
    model_space={basedih:basedih, slqs:slqs, slqs_cos:slqs_cos}
    results=[]
    y_trues=[]
    
    if model is not None:
        for pair in predict_pairs: # global
            hypo=pair[1]
            hyper=pair[0]
        
            y_trues.append(pair[-1])
        
            if metric_name is None:
                results.append(model_space[model].predict(hypo, hyper))
            
            else: results.append(model_space[model].predict(hypo, hyper, metric_name))
        
        return results, y_trues
    
    else: 
        print("Please specify the model!")
        
        for pair in predict_pairs: # global
            hypo=pair[1]
            hyper=pair[0]
        
            y_trues.append(pair[-1])
        return y_trues

    
y_trues = _predict()

pred_weeds, _ = _predict(basedih, "weeds_prec")
pred_clarkeDE, _ = _predict(basedih, "clarkeDE")
pred_invCL, _ = _predict(basedih, "invCL")
pred_cosine, _ = _predict(basedih, "cosine")

pred_slqs, _ = _predict(slqs)
pred_slqs_cos, _ = _predict(slqs_cos)

Please specify the model!


In [5]:
writer("weeds", pred_weeds)
writer("clarkeDE", pred_clarkeDE)
writer("invCL", pred_invCL)
writer("cosine", pred_cosine)
writer("slqs", pred_slqs)
writer("slqs_cos", pred_slqs_cos)

Evaluation

In [9]:
from eval import *
import pandas as pd

ap = [average_precision_score(y_trues, pred) for pred in 
      [pred_weeds, pred_clarkeDE, pred_invCL, pred_cosine, pred_slqs, pred_slqs_cos]]

ap_100 = [ap_at_k(y_trues, pred, 100) for pred in 
             [pred_weeds, pred_clarkeDE, pred_invCL, pred_cosine, pred_slqs, pred_slqs_cos]]

results = {"AP": ap, "AP@100": ap_100}
report = pd.DataFrame(results, columns=["AP","AP@100"], 
                      index=['weeds_prec', 'clarkeDE', 'invCL', 'cosine', 'slqs', 'slqs_cos'])

report.to_csv("./data/results.txt", sep="\t")
report.head(6)

,AP,AP@100
weeds_prec,0.546442,0.795885
clarkeDE,0.549216,0.801145
invCL,0.591873,0.854653
cosine,0.379949,0.494368
slqs,0.172089,0.182375
slqs_cos,0.314500,0.486050
